## eager execution dnn

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import logging
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', filename='./log/dnn_0515.log', level=logging.INFO)

import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

def parse_csv(line):
    a = [[0.0]] * 422
    b = [[0]]
    b = b + a
    example_defaults = b  # sets field types
    parsed_line = tf.decode_csv(line, example_defaults)
    # First 4 fields are features, combine into single tensor
    features = tf.reshape(parsed_line[1:], shape=(422,))
    # Last field is the label
    label = tf.reshape(parsed_line[0], shape=())
    return features, label

filenames = ["/home/mountain/atec/data/0514/train_feature00", "/home/mountain/atec/data/0514/train_feature01", "/home/mountain/atec/data/0514/train_feature02", "/home/mountain/atec/data/0514/train_feature03", "/home/mountain/atec/data/0514/train_feature04", "/home/mountain/atec/data/0514/train_feature05", "/home/mountain/atec/data/0514/train_feature06", "/home/mountain/atec/data/0514/train_feature07", "/home/mountain/atec/data/0514/train_feature08", "/home/mountain/atec/data/0514/train_feature09", "/home/mountain/atec/data/0514/train_feature10", "/home/mountain/atec/data/0514/train_feature11", "/home/mountain/atec/data/0514/train_feature12", "/home/mountain/atec/data/0514/train_feature13", "/home/mountain/atec/data/0514/train_feature14", "/home/mountain/atec/data/0514/train_feature15", "/home/mountain/atec/data/0514/train_feature16", "/home/mountain/atec/data/0514/train_feature17", "/home/mountain/atec/data/0514/train_feature18", "/home/mountain/atec/data/0514/train_feature19", "/home/mountain/atec/data/0514/train_feature20", "/home/mountain/atec/data/0514/train_feature21", "/home/mountain/atec/data/0514/train_feature22", "/home/mountain/atec/data/0514/train_feature23", "/home/mountain/atec/data/0514/train_feature24", "/home/mountain/atec/data/0514/train_feature25", "/home/mountain/atec/data/0514/train_feature26", "/home/mountain/atec/data/0514/train_feature27", "/home/mountain/atec/data/0514/train_feature28", "/home/mountain/atec/data/0514/train_feature29", "/home/mountain/atec/data/0514/train_feature30", "/home/mountain/atec/data/0514/train_feature31", "/home/mountain/atec/data/0514/train_feature32", "/home/mountain/atec/data/0514/train_feature33", "/home/mountain/atec/data/0514/train_feature34", "/home/mountain/atec/data/0514/train_feature35", "/home/mountain/atec/data/0514/train_feature36", "/home/mountain/atec/data/0514/train_feature37", "/home/mountain/atec/data/0514/train_feature38", "/home/mountain/atec/data/0514/train_feature39", "/home/mountain/atec/data/0514/train_feature40", "/home/mountain/atec/data/0514/train_feature41", "/home/mountain/atec/data/0514/train_feature42", "/home/mountain/atec/data/0514/train_feature43", "/home/mountain/atec/data/0514/train_feature44", "/home/mountain/atec/data/0514/train_feature45", "/home/mountain/atec/data/0514/train_feature46", "/home/mountain/atec/data/0514/train_feature47", "/home/mountain/atec/data/0514/train_feature48", "/home/mountain/atec/data/0514/train_feature49", "/home/mountain/atec/data/0514/train_feature50", "/home/mountain/atec/data/0514/train_feature51", "/home/mountain/atec/data/0514/train_feature52", "/home/mountain/atec/data/0514/train_feature53", "/home/mountain/atec/data/0514/train_feature54", "/home/mountain/atec/data/0514/train_feature55", "/home/mountain/atec/data/0514/train_feature56", "/home/mountain/atec/data/0514/train_feature57", "/home/mountain/atec/data/0514/train_feature58", "/home/mountain/atec/data/0514/train_feature59", "/home/mountain/atec/data/0514/train_feature60", "/home/mountain/atec/data/0514/train_feature61", "/home/mountain/atec/data/0514/train_feature62", "/home/mountain/atec/data/0514/train_feature63", "/home/mountain/atec/data/0514/train_feature64", "/home/mountain/atec/data/0514/train_feature65", "/home/mountain/atec/data/0514/train_feature66"]

train_dataset = tf.data.TextLineDataset(filenames)
train_dataset = train_dataset.map(parse_csv)      # parse each row
train_dataset = train_dataset.shuffle(buffer_size=10001)  # randomize
train_dataset = train_dataset.batch(256)

# View a single example entry from a batch
features, label = tfe.Iterator(train_dataset).next()

model = tf.keras.Sequential([
  tf.keras.layers.Dense(256, activation="relu", input_shape=(422,)),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

def loss(model, x, y):
    y_ = model(x)
    y = tf.reshape(y, [-1, 1])
    return tf.losses.log_loss(labels=y, predictions=y_)

def grad(model, inputs, targets):
    with tfe.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, model.variables)

optimizer = tf.train.AdamOptimizer(learning_rate=0.005)

prediction = []
label = []

num_epochs = 20

for epoch in range(num_epochs):
    logging.info("epoch: %s" % epoch)
#     print epoch
    epoch_loss_avg = tfe.metrics.Mean()

    i = 0
    for x, y in tfe.Iterator(train_dataset):
        # Optimize the model
        i += 1
        if i % 100 == 0:
            logging.info("i: %s" % i)
#             print i
        grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.variables),
                              global_step=tf.train.get_or_create_global_step())

        # Track progress
        epoch_loss_avg(loss(model, x, y))  # add current batch loss
        

    logging.info("Epoch {:03d}: Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))
#     print epoch_loss_avg.result()

checkpoint_prefix = "./model/dnn_0514"
root = tfe.Checkpoint(optimizer=optimizer,
                      model=model,
                      optimizer_step=tf.train.get_or_create_global_step())

root.save(file_prefix=checkpoint_prefix)
# or
# root.restore(tf.train.latest_checkpoint(checkpoint_dir))

训练时间
1. 一轮4个小时

## keras dnn（学习模仿）

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv1D,MaxPooling2D,Convolution1D
from keras.optimizers import SGD,Adam,RMSprop
from keras.layers import Dense, Activation, Dropout, Flatten, normalization
from keras.datasets import mnist
from keras import backend as K # turn image into speficifc shape
import pandas as pd
import numpy as np
import keras
# load data
train_labeled = pd.read_hdf("train_labeled.h5", "train")
train_unlabeled = pd.read_hdf("train_unlabeled.h5", "train")
test = pd.read_hdf("test.h5", "test")
y_train_original = train_labeled.values[2000:,0]
X_train_original = train_labeled.values[2000:,1:129]
y_val =  train_labeled.values[0:2000,0]
X_val =  train_labeled.values[0:2000,1:129]
X_unlabeled = train_unlabeled.values[:,0:128]
y_train_original = keras.utils.to_categorical(y_train_original,10)
y_val =  keras.utils.to_categorical(y_val,10)
X_valid = test.values[:,0:128]

model = Sequential()
model.add(normalization.BatchNormalization(input_shape=(128,)))
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(400, activation = 'relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

sgd = SGD(lr=0.01, momentum=0.9, decay=0.01, nesterov=True)
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
#adam = optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='categorical_crossentropy',
#               optimizer=sgd,
#               metrics=['accuracy'])
adam = Adam()
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

acc_last = 0
acc_now = 1

print("====last_time===")
model = Sequential()
model.add(normalization.BatchNormalization(input_shape=(128,)))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(400, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = Adam()
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train_original, y_train_original, epochs=50, verbose=0, batch_size=100)  # 461 epch
score = model.evaluate(X_val, y_val)
print("==acc last = first ===")
acc_last = score[1]
print(acc_last)

# predict
y_unlabelled = model.predict_proba(X_unlabeled, batch_size=32, verbose=0)
y_add_train = model.predict_classes(X_unlabeled)
y_max_p = y_unlabelled.max(axis=1)
indice = y_max_p > (np.mean(y_max_p)+2.58 * np.std(y_max_p)/(len(y_max_p)**0.5))
X_add_train = X_unlabeled[indice, :]
y_add_train = keras.utils.to_categorical(y_add_train[indice], 10)
X_train_original = np.concatenate((X_train_original, X_add_train), axis=0)
y_train_original = np.concatenate((y_train_original, y_add_train), axis=0)
print(y_train_original.shape)
X_unlabeled = X_unlabeled[~indice,:]


acc_now  =  acc_last
print("==acc now = first ===")
print(acc_now)
i = 0
while acc_now >= acc_last:
    # y_train_original = train_labeled.values[2000:, 0]
    # X_train_original = train_labeled.values[2000:, 1:129]
    acc_last = acc_now
    print("traing data size")
    print(y_train_original.shape)
    print("====update=====")
    model = Sequential()
    model.add(normalization.BatchNormalization(input_shape=(128,)))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(400, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    adam = Adam()
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train_original, y_train_original, epochs=50+10*i, verbose=0, batch_size=100)  # 461 epch
    score = model.evaluate(X_val, y_val)
    acc_now = score[1]

    # predict
    y_unlabelled = model.predict_proba(X_unlabeled, batch_size=32, verbose=0)
    y_add_train = model.predict_classes(X_unlabeled)
    y_max_p = y_unlabelled.max(axis=1)
    indice = y_max_p > (np.mean(y_max_p) + 2.58 * np.std(y_max_p)/(len(y_max_p**0.5)))
    X_add_train = X_unlabeled[indice, :]
    y_add_train = keras.utils.to_categorical(y_add_train[indice], 10)
    X_train_original = np.concatenate((X_train_original, X_add_train), axis=0)
    y_train_original = np.concatenate((y_train_original, y_add_train), axis=0)
    print("training data now")
    print(y_train_original.shape)
    if len(X_unlabeled[~indice, :]) == 0:
        break
    else:
        X_unlabeled = X_unlabeled[~indice, :]
    print("acc last---  acc now")
    print(acc_last, acc_now)
    if acc_now<=acc_last:
        break

    i = i + 1

    y_valid = model.predict_classes(X_valid)
    result = pd.DataFrame(columns=['Id', 'y'])
    result['y'] = np.transpose(y_valid)
    result['Id'] = range(30000, 38000)
    result.to_csv('labelled'+str(i)+'.csv', index=False, header=True)